In [8]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import colors as mcolors
from tqdm import tqdm
import gc
import warnings
warnings.filterwarnings('ignore')

# 1.0 Load data from association rules

In [2]:
# Load association results:

association_results_df = pd.read_csv('../data/20220504_table_association_results_wardrobesize_2-20_wo_none.csv', \
    index_col=0,  dtype={'antecedants':'string', 'consequents':'string'})

association_results_df.sort_values('lift', ascending=False, inplace=True)
association_results_df

,antecedants,consequents,support,confidence,lift
720,0882759003,0882757003,0.000116,0.808696,6260.392977
719,0874078003,0912418001,0.000114,0.844037,6177.581318
377,0698260005,0698255005,0.000108,0.813084,6117.888025
657,0833490002,0833486002,0.000101,0.743119,5982.853211
646,0845699001,0832298002,0.000119,0.727273,5740.463458
...,...,...,...,...,...
396,0706016003,0706016001,0.001151,0.248459,23.447922
395,0706016002,0706016001,0.002006,0.235870,22.259842
401,0706016025,0706016001,0.000127,0.235023,22.179938
399,0706016015,0706016001,0.000761,0.223315,21.075021


In [3]:
# Generate wardrobe table:
df_trans = pd.read_csv('../data/transactions_train.csv', dtype={'article_id':'string'})
# Train test split:
df_trans_train = df_trans.query('t_dat < "2020-09-16"').copy()
df_trans_red = df_trans_train.drop(columns=['t_dat', 'price', 'sales_channel_id']).copy()
df_wardrobe = df_trans_red.groupby('customer_id')['article_id'].aggregate(lambda x: list(x)).reset_index()


In [4]:
# Clear memory
del [[df_trans,df_trans_red, df_trans_train]]

In [5]:
df_wardrobe.head()


,customer_id,article_id
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,"[0625548001, 0176209023, 0627759010, 069713800..."
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,"[0583558001, 0639677008, 0640244003, 052126900..."
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,"[0663713001, 0541518023, 0663713001, 057802000..."
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,"[0742079001, 0732413001]"
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,"[0634249005, 0677049001, 0698286003, 070770400..."


In [54]:
# Add 'number of articles' column (propably needed to generate smaller dataset) and sort descending:

df_wardrobe['no_articles'] = df_wardrobe.article_id.apply(lambda x: len(x))
df_wardrobe.sort_values('no_articles', ascending=False, inplace=True)

In [6]:
# # Select a sample for testing code
# df_sample = df_wardrobe.sample(n=1000000, random_state=42)
# df_sample.sort_values('no_articles', ascending=False)

# Since no sample is necessary df_sample will be df_wardrobe:
df_sample = df_wardrobe.copy()

In [9]:
# Clear memory
del [[df_wardrobe]]

gc.collect()

NameError: name 'df_wardrobe' is not defined

# 2.0 Assign product recommendations based on current wardrobes

In [10]:
# Generate empty column for recommendation
df_sample['reco'] = df_sample.apply(lambda x: [], axis=1)

In [11]:
# Append consequents as list
# Info: needed 186 minutes for wardrobes between 2-20 products.

for i in range(len(df_sample)):
    for j in range(len(association_results_df)):
        if association_results_df.antecedants.iloc[j] in df_sample.article_id.iloc[i]:
            if association_results_df.consequents.iloc[j] not in df_sample.article_id.iloc[i]:
                df_sample['reco'].iloc[i].append(association_results_df.consequents.iloc[j])
            



In [12]:
df_sample['no_reco'] = df_sample.reco.apply(lambda x: len(x))
df_sample.sort_values('no_reco', ascending=False).head()


,customer_id,article_id,reco,no_reco
1007560,be1981ab818cf4ef6765b2ecaea7a2cbf14ccd6e8a7ee9...,"[0658506001, 0662980002, 0667709001, 068568700...","[0758060001, 0492244014, 0799410003, 078338000...",60
64883,0c44b5697c84f49c39712df9b5452ec131cf6cf4090e35...,"[0399136060, 0399136061, 0668390001, 042619900...","[0802980001, 0698283002, 0832284003, 054246400...",53
238923,2d17eb6bc25d969da89b83e3d446041c90bc99789998a3...,"[0680810001, 0537346022, 0550309001, 063967700...","[0758060001, 0843873003, 0740720001, 075800201...",45
561018,69dcbfa8db2b3b9832657dd3f693cdb9bae009b5e3c289...,"[0658298001, 0657291001, 0657291001, 063967700...","[0833486002, 0772032001, 0758064002, 060004301...",45
19815,03c3771e117f921c472552497e243607243fd7de0c57fd...,"[0688873005, 0618899009, 0670687001, 067068700...","[0871468001, 0861477001, 0767438001, 081183500...",44


In [55]:
# Number of customer_ids with at least X recommendations:
print("The dataset includes...")
print(f"- {df_sample[df_sample['no_reco']>0].shape[0]} customers with at least 1 recommendation")
print(f"- {df_sample[df_sample['no_reco']>5].shape[0]} customers with at least 6 recommendations")
print(f"- {df_sample[df_sample['no_reco']>12].shape[0]} customers with more than 12 recommendations")


The dataset includes...
- 541901 customers with at least 1 recommendation
- 111490 customers with at least 6 recommendations
- 23616 customers with more than 12 recommendations


In [ ]:
# sort consequents by lift-value
# ToDo!

NameError: name 'e' is not defined

In [24]:
# Create reco-column ("prediction") which stores recos in one string:
# df_sample['prediction'] = ' '.join(df_sample['reco'])
df_sample['prediction'] = df_sample.reco.apply(lambda x: ' '.join(x))

In [28]:
df_sample.head()

,customer_id,article_id,reco,no_reco,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,"[0625548001, 0176209023, 0627759010, 069713800...",[0568597007],1,0568597007
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,"[0583558001, 0639677008, 0640244003, 052126900...","[0776237020, 0688537005, 0589599036, 083835700...",9,0776237020 0688537005 0589599036 0838357002 04...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,"[0663713001, 0541518023, 0663713001, 057802000...","[0458543001, 0458543009, 0838055001, 0823505002]",4,0458543001 0458543009 0838055001 0823505002
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,"[0742079001, 0732413001]",[],0,
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,"[0634249005, 0677049001, 0698286003, 070770400...",[0692721005],1,0692721005


In [26]:
# Store recommendations as csv:
df_sample.to_csv('../data/20220504_test_all_cust_recommendations_wardrobesize_2-20_wo_none.csv')

# 3.0 Generate submission csv

In [41]:
# Generate submission train file (only customer_ids which exist in train dataset):
df_submission_train = df_sample.drop(columns=['article_id', 'reco', 'no_reco'])


In [47]:
df_submission_train.head()

,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0568597007
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0776237020 0688537005 0589599036 0838357002 04...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0458543001 0458543009 0838055001 0823505002
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0692721005


In [36]:
# Load sample_submission to get all customer_ids and drop sample predictions:
df_sample_sub = pd.read_csv('../data/sample_submission.csv')
df_sample_sub.drop(columns=['prediction'], inplace=True)
df_sample_sub.shape

In [48]:
# Generate full submission file with all customer_ids:
df_submission_test = pd.merge(df_sample_sub, df_submission_train, how= 'left', on="customer_id")


In [57]:
# Fill up with baseline-model recommendations:

df_submission_test_baseline = df_submission_test.copy()

In [68]:
baseline = " 0706016001 0706016002 0372860001 0610776002 0759871002 0464297007 0372860002 0610776001 0399223001 0706016003 0720125001 0156231001"
df_submission_test_baseline['prediction'] = df_submission_test_baseline['prediction'] + baseline
df_submission_test_baseline

,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0568597007 0706016001 0706016002 0372860001 06...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0776237020 0688537005 0589599036 0838357002 04...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0458543001 0458543009 0838055001 0823505002 07...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0706016001 0706016002 0372860001 0610776002 0...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0692721005 0706016001 0706016002 0372860001 06...
...,...,...
1371975,ffffbbf78b6eaac697a8a5dfbfd2bfa8113ee5b403e474...,0706016001 0706016002 0372860001 0610776002 0...
1371976,ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab5...,0854677001 0776237020 0838357002 0624486064 05...
1371977,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,0573716002 0573716050 0706016006 0706016006 07...
1371978,ffffd7744cebcf3aca44ae7049d2a94b87074c3d4ffe38...,0706016001 0706016002 0372860001 0610776002 0...


In [78]:
df_submission_test_baseline['prediction'].fillna(value=baseline, inplace=True)

In [80]:
df_submission_test_baseline.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1371980 entries, 0 to 1371979
Data columns (total 2 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   customer_id  1371980 non-null  object
 1   prediction   1371980 non-null  object
dtypes: object(2)
memory usage: 31.4+ MB


In [81]:
# Store submission file as csv:
df_submission_test_baseline.to_csv('../data/MBA_Baseline_20220504_submission_wardrobesize_2-20.csv', index = False)